In [1]:
import http.client
import json
from settings import API_KEY
import time
from pandas.io.json import json_normalize
import pandas as pd
import  csv
import ast

## Funções

In [2]:
def get_url_credits(movie_id, api_key):
    return("/3/movie/" + str(movie_id) + "/credits" + "?api_key=" + str(api_key))

def get_json(conn, url):
	payload = "{}"
	conn.request("GET", url, payload)
	res = conn.getresponse()
	data = res.read()
	return(json.loads(data.decode("    utf-8")))

In [3]:
def get_movie_credits(ids):
    
	conn = http.client.HTTPSConnection("api.themoviedb.org")    
	movie_credits_df = pd.DataFrame()
    
	for movie_id in ids:
		url = get_url_credits(movie_id, API_KEY)
		movie_data = get_json(conn, url)
        
        #mudei!
		if ('status_code' in movie_data):
			time.sleep(10)  
			movie_data = get_json(conn, url)
		#else:
		#	movie_data = get_json(conn, url)
        
		df = json_normalize(movie_data)
		movie_credits_df = movie_credits_df.append(df)

	return movie_credits_df

In [10]:
def get_cast(ids, movie_credits_df):

    movie_cast_df = pd.DataFrame()

    #print(movie_credits_df['id'])
    #for idx,movie_id in enumerate(ids,40):
    for movie_id in ids_num:

        movie_item = pd.DataFrame()
        #print(movie_credits_df.head(90))
        #print("movie_id: %s" % (movie_id))
        movie_item = movie_credits_df[movie_credits_df['id']==movie_id].reset_index()
        #print(movie_item.head(90))
        #print("%s of %s" % (idx,len(ids)))

        #print(movie_item['cast'])
        
        for cast_id in movie_item['cast'][0]:
            cast_id.update({'movie_id': int(movie_id)})
            cast_id.pop('profile_path', None)
            movie_cast_df = movie_cast_df.append(cast_id, ignore_index=True)
    
    return movie_cast_df

## Testes

In [6]:
with open('out_sample.csv', 'r') as f:
#with open('out_all.csv', 'r') as f:    
    reader = csv.reader(f, skipinitialspace=True, delimiter=',')
    ids_list = list(reader)
    
ids = [item for sublist in ids_list for item in sublist]

In [7]:
len(ids)

156

In [8]:
# Transformando ids em numérico
ids_num = list(map(int, ids))

In [12]:
start_time = time.time()
movie_credits_df = get_movie_credits(ids_num)
print("--- Movie Credits: %s seconds ---" % (time.time() - start_time))
print("")

--- Movie Credits: 58.991355895996094 seconds ---



In [14]:
movie_credits_df = movie_credits_df.reset_index().drop('index', axis=1)

In [15]:
movie_credits_df.shape

(156, 3)

In [16]:
movie_credits_df.head()

,cast,crew,id
0,[{'profile_path': '/qB0BGUhMph48P1msl9wHRgL4Ae...,[{'profile_path': '/7OGmfDF4VHLLgbjxuEwTj3ga0u...,77
1,[{'profile_path': '/ktuOAyUHM2u2YPPChH8gAArxJ7...,[{'profile_path': '/oTAL0z0vsjipCruxXUsDUIieuh...,98
2,[{'profile_path': '/2ocrTd8ChuUyJOzMuD4PsQw8eC...,[{'profile_path': '/qlWi0kufzB7lCq6Jz5zuNE3Kk1...,1359
3,[{'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlh...,[{'profile_path': '/isCuZ9PWIOyXzdf3ihodXzjIum...,8358
4,[{'profile_path': '/6VGgL0bBvPIJ9vDOyyGf5nK2zL...,[{'profile_path': '/fBaZLXX3rBwhSXvtplQEtvW7VS...,7443


In [17]:
movie_credits_df['id'].isnull().sum()

0

In [18]:
movie_credits_df.duplicated('id').sum()

0

In [19]:
#from IPython.display import display
#pd.options.display.max_rows = None
#movie_credits_df.head(90)

In [20]:
start_time = time.time()
movie_cast_df = get_cast(ids_num, movie_credits_df)
print("--- Movie Cast: %s seconds ---" % (time.time() - start_time))
print("")

--- Movie Cast: 15.14481806755066 seconds ---



In [21]:
movie_cast_df.head()

,cast_id,character,credit_id,gender,id,movie_id,name,order
0,4.0,Leonard,52fe4214c3a36847f80024db,2.0,529.0,77.0,Guy Pearce,0.0
1,5.0,Natalie,52fe4214c3a36847f80024df,1.0,530.0,77.0,Carrie-Anne Moss,1.0
2,6.0,Teddy Gammell,52fe4214c3a36847f80024e3,2.0,532.0,77.0,Joe Pantoliano,2.0
3,7.0,Burt,52fe4214c3a36847f80024e7,2.0,534.0,77.0,Mark Boone Junior,3.0
4,10.0,Sammy,52fe4214c3a36847f80024f3,2.0,537.0,77.0,Stephen Tobolowsky,4.0
